Build the First Version of the Recommender
Users shall input their preferred music type (e.g., Jazz Fusion), and the system shall return a song from that category.
Additionally, if the user asks for a "currently popular song," your system will recommend one from the trending songs list that you scraped earlier.

User Song Type Selection (Basic Recommender Prototype)
Create a program where the user can input a music type rather than a specific song.
The user can choose from: "High Energy", "Chill Vibes", "Rock", "Trending Now", etc.
For now, if the user selects "Trending Now," return a random popular song from the Billboard Hot 100 DataFrame.
If they select another type, simply acknowledge their selection (without returning a song yet).

In [201]:
# Import config file
import config

In [276]:
# Import libraries

import spotipy
import pandas as pd
import random
import json
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import pprint
from IPython.display import IFrame
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import difflib
import time
from IPython.display import display

In [148]:
# load 100 top Billboard songs dataset 
songs_billboard = "df_song_dict_billboard.csv"
df_songs_billboard = pd.read_csv(songs_billboard)
df_songs_billboard

,title,artist,rank
0,Luther,Kendrick Lamar & SZA,1
1,Die With A Smile,Lady Gaga & Bruno Mars,2
2,Not Like Us,Kendrick Lamar,3
3,TV Off,Kendrick Lamar Featuring Lefty Gunplay,4
4,A Bar Song (Tipsy),Shaboozey,5
...,...,...,...
95,No Pole,Don Toliver,96
96,Small Town Fame,Drake,97
97,Que Pasaria...,Rauw Alejandro & Bad Bunny,98
98,Greenlight,Tate McRae,99


In [150]:
df_songs_billboard.sample()["title"]

62    Good News
Name: title, dtype: object

In [185]:
# Get a random song from df_songs_billboard

def get_random_song(df_songs_billboard):
    return df_songs_billboard.sample(n=1).iloc[0]

In [205]:
# Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [187]:
# Retrieving Track_ID for songs from Spotify

def get_track_id_from_spotify(song_name, artist_name):
    # Search for the song on Spotify
    query = f"{song_name} {artist_name}"
    results = sp.search(query, limit=1, type='track', market='US')
    
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        track_id = track['id']
        print(f"Track ID for '{song_name}' by {artist_name}: {track_id}")
        return track_id
    else:
        print("Sorry, the song was not found on Spotify.")
        return None

In [161]:
# Create categories of different music types to choose from:

def music_type_selection():
    track_id = None  # Initialize track_id to avoid referencing it before assignment
    print("Please select a music type:")
    print("1. High Energy")
    print("2. Chill Vibes")
    print("3. Rock")
    print("4. Trending Now")

    selection = input("Enter the number corresponding to your choice: ")

    if selection == "4":
        # Random song from df_songs_billboard for "Trending Now"
        song = get_random_song(df_songs_billboard)
        print(f"Here is a random trending song: {song['title']} by {song['artist']}")
        # Play song using the embedded player:
        track_id=get_track_id_from_spotify(song['title'], song['artist'])
        
    elif selection in ["1", "2", "3"]:
        # Acknowledge the selection for other types
        if selection == "1":
            print("You selected 'High Energy'. Enjoy the beats!")
        elif selection == "2":
            print("You selected 'Chill Vibes'. Relax and unwind!")
        elif selection == "3":
            print("You selected 'Rock'. Let the music roll!")
    else:
        print("Invalid selection. Please choose a valid option.")

    # Play a song according to Spotify track ID jusing the embedded player
    def play_song(track_id):
        if track_id:  # Only create the player if track_id is valid
            return IFrame(src=f"https://open.spotify.com/embed/track/{track_id}",
                      width="320",
                      height="80",
                      frameborder="0",
                      allowtransparency="true",
                      allow="encrypted-media")
        else:
            print("Invalid track ID. Cannot play song.")
            return None

    return play_song(track_id)

# Call the function to prompt user for music selection
music_type_selection()

Please select a music type:
1. High Energy
2. Chill Vibes
3. Rock
4. Trending Now


Enter the number corresponding to your choice:  4


Here is a random trending song: Please Please Please by Sabrina Carpenter
Track ID for 'Please Please Please' by Sabrina Carpenter: 5N3hjp1WNayUPZrA8kJmJP


In [167]:
# Function to match user input with predefined options using difflib
def get_closest_match(user_input, options):
    # Use difflib to find close matches
    close_matches = difflib.get_close_matches(user_input, options, n=1, cutoff=0.6)
    return close_matches[0] if close_matches else None

# Create categories of different music types to choose from:
def music_type_selection():
    track_id = None  # Initialize track_id to avoid referencing it before assignment
    options = ["High Energy", "Chill Vibes", "Rock", "Trending Now"]  # List of options
    print("Please select a music type:")
    print("1. High Energy")
    print("2. Chill Vibes")
    print("3. Rock")
    print("4. Trending Now")

    # Adding difflib string matching feature for music type selection
    user_input = input("Enter a music type (e.g., 'Chill Vibes'): ")
    match = get_closest_match(user_input, options)  # Find the closest match

    if match:
        print(f"You selected '{match}'. Enjoy the music!")
        # Play song depending on match
        if match == "Trending Now":
            song = get_random_song(df_songs_billboard)
            print(f"Here is a random trending song: {song['title']} by {song['artist']}")
            track_id = get_track_id_from_spotify(song['title'], song['artist'])
        elif match == "High Energy":
            print("You selected 'High Energy'. Enjoy the beats!")
        elif match == "Chill Vibes":
            print("You selected 'Chill Vibes'. Relax and unwind!")
        elif match == "Rock":
            print("You selected 'Rock'. Let the music roll!")
    else:
        print("Could not match your input to any music type.")

    # Play a song according to Spotify track ID using the embedded player
    def play_song(track_id):
        if track_id:  # Only create the player if track_id is valid
            return IFrame(src=f"https://open.spotify.com/embed/track/{track_id}",
                          width="320",
                          height="80",
                          frameborder="0",
                          allowtransparency="true",
                          allow="encrypted-media")
        else:
            print("Invalid track ID. Cannot play song.")
            return None

    return play_song(track_id)

# Call the function to prompt user for music selection
music_type_selection()

Please select a music type:
1. High Energy
2. Chill Vibes
3. Rock
4. Trending Now


Enter a music type (e.g., 'Chill Vibes'):  Trending Now


You selected 'Trending Now'. Enjoy the music!
Here is a random trending song: I Know Love by Tate McRae Featauring The Kid LAROI
Track ID for 'I Know Love' by Tate McRae Featauring The Kid LAROI: 1YHQQFtKqFSFrqL4cNvstU


Final Song Recommendation System - Preparations

In [175]:
# Add cluster number and cluster name to the Billboard:

# Add a new column 'Cluster' with all values set to 8
df_songs_billboard['Cluster'] = 8

# Add a new column 'Cluster_Name' with all values set to "Trending 100 Songs"
df_songs_billboard['Cluster_Name'] = "Trending 100 Songs"

df_songs_billboard.head(20)

,title,artist,rank,Cluster,Cluster_Name
0,Luther,Kendrick Lamar & SZA,1,8,Trending 100 Songs
1,Die With A Smile,Lady Gaga & Bruno Mars,2,8,Trending 100 Songs
2,Not Like Us,Kendrick Lamar,3,8,Trending 100 Songs
3,TV Off,Kendrick Lamar Featuring Lefty Gunplay,4,8,Trending 100 Songs
4,A Bar Song (Tipsy),Shaboozey,5,8,Trending 100 Songs
5,APT.,ROSE & Bruno Mars,6,8,Trending 100 Songs
6,Birds Of A Feather,Billie Eilish,7,8,Trending 100 Songs
7,Pink Pony Club,Chappell Roan,8,8,Trending 100 Songs
8,Lose Control,Teddy Swims,9,8,Trending 100 Songs
9,Squabble Up,Kendrick Lamar,10,8,Trending 100 Songs


In [171]:
# load clustered audio features dataset 
audio_features_clustered = "df_audio_features_all_clustered_kmeans8.csv"
df_audio_features_clustered = pd.read_csv(audio_features_clustered)
df_audio_features_clustered

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,Cluster,Cluster_Name
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,...,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,5,Cheerful & Energetic Dance
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,...,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,3,Acoustic & Melancholic Indipendent
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,...,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,6,Vocal & Melodic Indipendent
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,...,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,3,Acoustic & Melancholic Indipendent
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,...,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,6,Vocal & Melodic Indipendent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90455,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,5,...,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,3,Acoustic & Melancholic Indipendent
90456,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,0,...,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,0,Slow & Melancholic Vibes
90457,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,0,...,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,0,Slow & Melancholic Vibes
90458,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,7,...,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,2,Positive & Melodic Pop Vibes


In [179]:
# Change the column names of "df_audio_features_clustered" dataframe to match with Billboard dataframe
df_audio_features_clustered.rename(columns={"track_name": "title", "artists": "artist"}, inplace=True)
df_audio_features_clustered

,track_id,artist,album_name,title,popularity,duration_ms,explicit,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,Cluster,Cluster_Name
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,...,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,5,Cheerful & Energetic Dance
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,...,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,3,Acoustic & Melancholic Indipendent
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,...,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,6,Vocal & Melodic Indipendent
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,...,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,3,Acoustic & Melancholic Indipendent
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,...,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,6,Vocal & Melodic Indipendent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90455,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,5,...,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,3,Acoustic & Melancholic Indipendent
90456,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,0,...,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,0,Slow & Melancholic Vibes
90457,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,0,...,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,0,Slow & Melancholic Vibes
90458,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,7,...,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,2,Positive & Melodic Pop Vibes


In [217]:
# Create a list for cluster naming
cluster_list = [
    "Slow & Melancholic Vibes",
    "Hard & Energetic", 
    "Positive & Melodic Pop Vibes", 
    "Acoustic & Melancholic Indipendent", 
    "Energetic Beats", 
    "Cheerful & Energetic Dance",
    "Vocal & Melodic Indipendent", 
    "Explicit & Verbal", 
    "Trending 100 Songs"]
cluster_list

['Slow & Melancholic Vibes',
 'Hard & Energetic',
 'Positive & Melodic Pop Vibes',
 'Acoustic & Melancholic Indipendent',
 'Energetic Beats',
 'Cheerful & Energetic Dance',
 'Vocal & Melodic Indipendent',
 'Explicit & Verbal',
 'Trending 100 Songs']

In [219]:
# Create a dictionary for cluster naming
cluster_mapping = {
    0: "Slow & Melancholic Vibes",
    1: "Hard & Energetic", 
    2: "Positive & Melodic Pop Vibes", 
    3: "Acoustic & Melancholic Indipendent", 
    4: "Energetic Beats", 
    5: "Cheerful & Energetic Dance",
    6: "Vocal & Melodic Indipendent", 
    7: "Explicit & Verbal", 
    8: "Trending 100 Songs"}
cluster_mapping

{0: 'Slow & Melancholic Vibes',
 1: 'Hard & Energetic',
 2: 'Positive & Melodic Pop Vibes',
 3: 'Acoustic & Melancholic Indipendent',
 4: 'Energetic Beats',
 5: 'Cheerful & Energetic Dance',
 6: 'Vocal & Melodic Indipendent',
 7: 'Explicit & Verbal',
 8: 'Trending 100 Songs'}

In [209]:
# Retrieving Track_ID for all songs in the "df_songs_billboard" dataframe from Spotify
# And adding them to the "df_songs_billboard" dataframe

def get_track_id_from_spotify(title, artist):
    # Search for the song on Spotify using the column names in the Billboard dataframe
    query = f"{title} {artist}"
    results = sp.search(query, limit=1, type='track', market='US')
    
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        track_id = track['id']
        print(f"Track ID for '{title}' by {artist}: {track_id}")
        return track_id
    else:
        print(f"Sorry, the song '{title}' by {artist} was not found on Spotify.")
        return None

# Apply the "get_track_id_from_spotify" function to the "df_songs_billboard" dataframe
def fetch_track_ids(df_songs_billboard):
    df_songs_billboard['track_id'] = None  # Initialize the track_id column

    for index, row in df_songs_billboard.iterrows():
        title, artist = row['title'], row['artist']
        df_songs_billboard.at[index, 'track_id'] = get_track_id_from_spotify(title, artist)
        time.sleep(0.5)  # Avoid hitting rate limits

    return df_songs_billboard

fetch_track_ids(df_songs_billboard)

Track ID for 'Luther' by Kendrick Lamar & SZA: 2CGNAOSuO1MEFCbBRgUzjd
Track ID for 'Die With A Smile' by Lady Gaga & Bruno Mars: 2plbrEY59IikOBgBGLjaoe
Track ID for 'Not Like Us' by Kendrick Lamar: 6AI3ezQ4o3HUoP6Dhudph3
Track ID for 'TV Off' by Kendrick Lamar Featuring Lefty Gunplay: 6g6W3pMAFbp2FS1QCijkpM
Track ID for 'A Bar Song (Tipsy)' by Shaboozey: 2FQrifJ1N335Ljm3TjTVVf
Track ID for 'APT.' by ROSE & Bruno Mars: 5vNRhkKd0yEAg8suGBpjeY
Track ID for 'Birds Of A Feather' by Billie Eilish: 6dOtVTDdiauQNBQEDOtlAB
Track ID for 'Pink Pony Club' by Chappell Roan: 1k2pQc5i348DCHwbn5KTdc
Track ID for 'Lose Control' by Teddy Swims: 6usohdchdzW9oML7VC4Uhk
Track ID for 'Squabble Up' by Kendrick Lamar: 0nj9Bq5sHDiTxSHunhgkFb
Track ID for 'Nokia' by Drake: 2u9S9JJ6hTZS3Vf22HOZKg
Track ID for 'Beautiful Things' by Benson Boone: 6tNQ70jh4OwmPGpYy6R2o9
Track ID for 'That's So True' by Gracie Abrams: 7ne4VBA60CxGM75vw0EYad
Track ID for 'Love Somebody' by Morgan Wallen: 7hR22TOX3RorxJPcsz5Wbo
Track 

,title,artist,rank,Cluster,Cluster_Name,track_id
0,Luther,Kendrick Lamar & SZA,1,8,Trending 100 Songs,2CGNAOSuO1MEFCbBRgUzjd
1,Die With A Smile,Lady Gaga & Bruno Mars,2,8,Trending 100 Songs,2plbrEY59IikOBgBGLjaoe
2,Not Like Us,Kendrick Lamar,3,8,Trending 100 Songs,6AI3ezQ4o3HUoP6Dhudph3
3,TV Off,Kendrick Lamar Featuring Lefty Gunplay,4,8,Trending 100 Songs,6g6W3pMAFbp2FS1QCijkpM
4,A Bar Song (Tipsy),Shaboozey,5,8,Trending 100 Songs,2FQrifJ1N335Ljm3TjTVVf
...,...,...,...,...,...,...
95,No Pole,Don Toliver,96,8,Trending 100 Songs,0eaVIYo2zeOaGJeqZ5TwYz
96,Small Town Fame,Drake,97,8,Trending 100 Songs,1YHQQFtKqFSFrqL4cNvstU
97,Que Pasaria...,Rauw Alejandro & Bad Bunny,98,8,Trending 100 Songs,0l0vcZMU7AOeQmUIREoI2U
98,Greenlight,Tate McRae,99,8,Trending 100 Songs,56LQyaXsYWQZ2b7OPLAKEO


In [211]:
df_songs_billboard

,title,artist,rank,Cluster,Cluster_Name,track_id
0,Luther,Kendrick Lamar & SZA,1,8,Trending 100 Songs,2CGNAOSuO1MEFCbBRgUzjd
1,Die With A Smile,Lady Gaga & Bruno Mars,2,8,Trending 100 Songs,2plbrEY59IikOBgBGLjaoe
2,Not Like Us,Kendrick Lamar,3,8,Trending 100 Songs,6AI3ezQ4o3HUoP6Dhudph3
3,TV Off,Kendrick Lamar Featuring Lefty Gunplay,4,8,Trending 100 Songs,6g6W3pMAFbp2FS1QCijkpM
4,A Bar Song (Tipsy),Shaboozey,5,8,Trending 100 Songs,2FQrifJ1N335Ljm3TjTVVf
...,...,...,...,...,...,...
95,No Pole,Don Toliver,96,8,Trending 100 Songs,0eaVIYo2zeOaGJeqZ5TwYz
96,Small Town Fame,Drake,97,8,Trending 100 Songs,1YHQQFtKqFSFrqL4cNvstU
97,Que Pasaria...,Rauw Alejandro & Bad Bunny,98,8,Trending 100 Songs,0l0vcZMU7AOeQmUIREoI2U
98,Greenlight,Tate McRae,99,8,Trending 100 Songs,56LQyaXsYWQZ2b7OPLAKEO


In [215]:
# Safe the updated datafile for df_songs_billboard dataframe:
# df_songs_billboard.to_csv("df_song_dict_billboard_ids.csv", index=False)
# ONLY DONE ONCE !

In [249]:
df_songs_billboard["Cluster"]=df_songs_billboard["Cluster"].astype(str)

In [251]:
# Summary of the data: checking data types
column_summary_df_songs_billboard = pd.DataFrame({
    'Column Name': df_songs_billboard.columns,
    'Data Type': df_songs_billboard.dtypes.values,
})

print(column_summary_df_songs_billboard)

    Column Name Data Type
0         title    object
1        artist    object
2          rank     int64
3       Cluster    object
4  Cluster_Name    object
5      track_id    object


In [253]:
df_audio_features_clustered["Cluster"]=df_audio_features_clustered["Cluster"].astype(str)

In [255]:
# Summary of the data: checking data types
column_summary_df_audio_features_clustered = pd.DataFrame({
    'Column Name': df_audio_features_clustered.columns,
    'Data Type': df_audio_features_clustered.dtypes.values,
})

print(column_summary_df_audio_features_clustered)

         Column Name Data Type
0           track_id    object
1             artist    object
2         album_name    object
3              title    object
4         popularity     int64
5        duration_ms     int64
6           explicit      bool
7       danceability   float64
8             energy   float64
9                key     int64
10          loudness   float64
11              mode     int64
12       speechiness   float64
13      acousticness   float64
14  instrumentalness   float64
15          liveness   float64
16           valence   float64
17             tempo   float64
18    time_signature     int64
19           Cluster    object
20      Cluster_Name    object


Final Song Recommendation System

In [269]:
# Create categories of different music types to choose from:

cluster_numbers = ["0", "1", "2", "3", "4", "5", "6", "7", "8"]
cluster_list = ["Slow & Melancholic Vibes", "Hard & Energetic", "Positive & Melodic Pop Vibes", "Acoustic & Melancholic Indipendent", 
                "Energetic Beats", "Cheerful & Energetic Dance", "Vocal & Melodic Indipendent", "Explicit & Verbal", "Trending 100 Songs"]
cluster_mapping = {"0": "Slow & Melancholic Vibes", "1": "Hard & Energetic", "2": "Positive & Melodic Pop Vibes", "3": "Acoustic & Melancholic Indipendent", 
                   "4": "Energetic Beats", "5": "Cheerful & Energetic Dance", "6": "Vocal & Melodic Indipendent", "7": "Explicit & Verbal", "8": "Trending 100 Songs"}

# Get a random song from df_songs_billboard or df_audio_features_clustered:
def get_random_song(songlist):
    return songlist.sample(n=1).iloc[0]

# Ask user to choose a category of music:
def music_type_selection():
    track_id = None  # Initialize track_id to avoid referencing it before assignment
    print("Please select a number corresponding to a music type:")
    print("0 - Slow & Melancholic Vibes")
    print("1 - Hard & Energetic")
    print("2 - Positive & Melodic Pop Vibes")
    print("3 - Acoustic & Melancholic Indipendent")
    print("4 - Energetic Beats")
    print("5 - Cheerful & Energetic Dance")
    print("6 - Vocal & Melodic Indipendent")
    print("7 - Explicit & Verbal")
    print("8 - Trending 100 Songs")

    selection = input("Please enter the number corresponding to your choice of music: ")

    while selection not in cluster_numbers:
        selection=input("Please enter a VALID number (0 - 8) corresponding to your choice of music: ")
    else:
        print(f"{selection} is corresponding to the music category {cluster_mapping[selection]}.")
        if selection == "8":
            # Random song from df_songs_billboard for "Trending 100 Songs"
            songlist = df_songs_billboard
            song = get_random_song(df_songs_billboard)
            print(f"Here is a random trending song: {song['title']} by {song['artist']} from {song['Cluster_Name']}")
            # Play song using the embedded player:
            track_id=song['track_id']
        
        elif selection in ["0", "1", "2", "3", "4", "5", "6", "7"]:
            # Random song from df_audio_features_clustered for corresponding cluster
            songlist = df_audio_features_clustered[df_audio_features_clustered["Cluster"]==selection]
            song = get_random_song(df_audio_features_clustered[df_audio_features_clustered["Cluster"]==selection])
            print(f"Here is a random song from the category {cluster_mapping[selection]}: {song['title']} by {song['artist']}")
            # Play song using the embedded player:
            track_id=song['track_id']
        
        else:
            print("Invalid selection. Please choose a valid number option.")

    # Play a song according to Spotify track ID using the embedded player
    def play_song(track_id):
        if track_id:  # Only create the player if track_id is valid
            return IFrame(src=f"https://open.spotify.com/embed/track/{track_id}",
                      width="320",
                      height="80",
                      frameborder="0",
                      allowtransparency="true",
                      allow="encrypted-media")
        else:
            print("Invalid track ID. Cannot play song.")
            return None

    return play_song(track_id)

# Call the function to prompt user for music selection
music_type_selection()

Please select a number corresponding to a music type:
0 - Slow & Melancholic Vibes
1 - Hard & Energetic
2 - Positive & Melodic Pop Vibes
3 - Acoustic & Melancholic Indipendent
4 - Energetic Beats
5 - Cheerful & Energetic Dance
6 - Vocal & Melodic Indipendent
7 - Explicit & Verbal
8 - Trending 100 Songs


Please enter the number corresponding to your choice of music:  4


4 is corresponding to the music category Energetic Beats.
Here is a random song from the category Energetic Beats: Ikebana by Nora Van Elken


In [271]:
# Check if the song was selected correctly from the df_audio_features_clustered dataframe:
print(df_audio_features_clustered[df_audio_features_clustered["title"]=="Ikebana"])

                     track_id          artist album_name    title  popularity  \
22641  4cYMt6WiUFShfZ6M1A9LPi  Nora Van Elken    Ikebana  Ikebana          47   

       duration_ms  explicit  danceability  energy  key  ...  mode  \
22641       169830     False         0.713   0.715   10  ...     0   

       speechiness  acousticness  instrumentalness  liveness  valence  \
22641       0.0266        0.0557              0.84     0.111     0.13   

         tempo  time_signature  Cluster     Cluster_Name  
22641  117.975               4        4  Energetic Beats  

[1 rows x 21 columns]


In [278]:
# Change the music_type_selector to recommend three songs instead of one

def get_random_songs(songlist, count=3):
    """Get a list of random songs from a DataFrame."""
    return songlist.sample(n=min(count, len(songlist)))

def music_type_selections():
    track_ids = []  # List to store track IDs
    print("Please select a number corresponding to a music type:")
    print("0 - Slow & Melancholic Vibes")
    print("1 - Hard & Energetic")
    print("2 - Positive & Melodic Pop Vibes")
    print("3 - Acoustic & Melancholic Independent")
    print("4 - Energetic Beats")
    print("5 - Cheerful & Energetic Dance")
    print("6 - Vocal & Melodic Independent")
    print("7 - Explicit & Verbal")
    print("8 - Trending 100 Songs")
    
    selection = input("Please enter the number corresponding to your choice of music: ")
    
    while selection not in cluster_numbers:
        selection = input("Please enter a VALID number (0 - 8) corresponding to your choice of music: ")
    
    print(f"{selection} corresponds to the music category {cluster_mapping[selection]}.")
    
    if selection == "8":
        songlist = df_songs_billboard
        songs = get_random_songs(songlist)
    else:
        songlist = df_audio_features_clustered[df_audio_features_clustered["Cluster"] == selection]
        songs = get_random_songs(songlist)
    
    for _, song in songs.iterrows():
        print(f"Here is a song from the category {cluster_mapping[selection]}: {song['title']} by {song['artist']}")
        track_ids.append(song['track_id'])
    
    def play_songs(track_ids):
        """Create and display embedded players for multiple songs."""
        if track_ids:
            players = [
                IFrame(
                    src=f"https://open.spotify.com/embed/track/{track_id}",
                    width="320",
                    height="80",
                    frameborder="0",
                    allowtransparency="true",
                    allow="encrypted-media"
                ) for track_id in track_ids
            ]
            for player in players:
                display(player)  # Explicitly display each player
        else:
            print("No valid track IDs. Cannot play songs.")
    
    return play_songs(track_ids)

# Call the function
music_type_selections()

Please select a number corresponding to a music type:
0 - Slow & Melancholic Vibes
1 - Hard & Energetic
2 - Positive & Melodic Pop Vibes
3 - Acoustic & Melancholic Independent
4 - Energetic Beats
5 - Cheerful & Energetic Dance
6 - Vocal & Melodic Independent
7 - Explicit & Verbal
8 - Trending 100 Songs


Please enter the number corresponding to your choice of music:  6


6 corresponds to the music category Vocal & Melodic Indipendent.
Here is a song from the category Vocal & Melodic Indipendent: That's How Much I Love You by Toini & The Tomcats
Here is a song from the category Vocal & Melodic Indipendent: Skintight by The Donnas
Here is a song from the category Vocal & Melodic Indipendent: Smoke On The Water - Remastered 2012 by Deep Purple
